# DeepSeek Agent 实战：小红书爆款文案生成助手

本 Notebook 将指导您如何使用 DeepSeek LLM 构建一个能够生成小红书爆款文案的智能 Agent。我们将从需求拆解开始，逐步定义 Agent 的系统提示词 (System Prompt)、外部工具 (Tools)，并实现其核心的工作流程，最终生成符合小红书平台特点的文案。

## 1. 环境准备与DeepSeek API配置

In [1]:
import os
from openai import OpenAI

# 建议将 API Key 设置为环境变量，避免直接暴露在代码中
# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("SILICON_API_KEY")
if not api_key:
    raise ValueError("请设置 SILICON_API_KEY 环境变量")

# 初始化 DeepSeek 客户端
client = OpenAI(
    api_key=api_key,
    base_url="https://api.siliconflow.cn/v1",  # DeepSeek API 的基地址
)

## 2. 需求拆解与Agent任务规划

#### 用户痛点与核心需求：
*   **效率低下：** 人工创作周期长，难以满足高频发布需求。
*   **创意瓶颈：** 难以持续产出新颖、吸引人的爆款创意。
*   **趋势捕捉难：** 实时流行元素难以快速融入文案。
*   **平台特性把握：** 小红书特有风格（标题、正文、标签、表情）难以精准复制。

#### “爆款”文案的特征：
*   **强吸引力标题：** 制造好奇、痛点共鸣、利益点清晰。
*   **沉浸式正文：** 真实体验分享、细节描述、情感共鸣。
*   **精准且多样标签：** 热门话题、品牌词、产品词、垂直领域词。
*   **生动表情符号：** 增强表达力，提升活泼感。
*   **清晰的行动召唤 (CTA)。**

#### Agent 任务规划：核心工作流
1.  **用户指令接收：** 接收产品信息、主题、风格等。
2.  **信息收集 (Web Search/DB Query)：** 实时搜索行业趋势、热门话题、竞品分析、产品卖点。
3.  **内容构思与初稿生成 (LLM)：** 结合所有信息，撰写标题、正文、标签、表情符号。
4.  **风格与格式优化 (LLM)：** 根据小红书平台特点和指定风格，对文案进行润色和结构调整。
5.  **最终输出：** 呈现完整文案。

## 3. 爆款文案生成逻辑与 Prompt 设计

### 3.1 System Prompt (系统提示词)

System Prompt 是 Agent 的“大脑”和“行为准则”。它定义了 Agent 的角色、目标以及工作方式。我们将采用 `Thought-Action-Observation` (ReAct) 模式来引导 DeepSeek 的推理过程。

In [2]:
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

### 3.2 Tools (工具定义)

Agent 的“双手”由一系列可调用的工具组成。这些工具扩展了 LLM 的能力，使其能够获取实时信息、查询数据库或执行特定操作。在这里，我们定义了三个核心工具：

*   `search_web`: 用于搜索互联网上的实时信息，如最新趋势、用户评价等。
*   `query_product_database`: 用于查询产品数据库，获取产品的详细卖点和特点。**此工具为模拟**。
*   `generate_emoji`: 用于根据文案内容生成恰当的表情符号。**此工具为模拟**。

In [3]:
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新新闻、流行趋势、用户评价、行业报告等。请确保搜索关键词精确，避免宽泛的查询。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "要搜索的关键词或问题，例如'最新小红书美妆趋势'或'深海蓝藻保湿面膜 用户评价'"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询内部产品数据库，获取指定产品的详细卖点、成分、适用人群、使用方法等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {
                        "type": "string",
                        "description": "要查询的产品名称，例如'深海蓝藻保湿面膜'"
                    }
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据提供的文本内容，生成一组适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {
                        "type": "string",
                        "description": "文案的关键内容或情感，例如'惊喜效果'、'补水保湿'"
                    }
                },
                "required": ["context"]
            }
        }
    }
]

### 3.3 模拟工具实现

由于我们无法直接调用真实的外部 API (如Google Search或内部产品数据库)，我们将创建一些模拟 (Mock) 工具函数来演示 Agent 的工作流程。在实际应用中，您需要将这些模拟函数替换为真实的 API 调用。

In [4]:
import random # 用于模拟生成表情
import time # 用于模拟网络延迟

def mock_search_web(query: str) -> str:
    """模拟网页搜索工具，返回预设的搜索结果。"""
    print(f"[Tool Call] 模拟搜索网页：{query}")
    time.sleep(1) # 模拟网络延迟
    if "小红书美妆趋势" in query:
        return "近期小红书美妆流行'多巴胺穿搭'、'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。用户痛点：卡粉、泛红、紧绷感。"
    elif "深海蓝藻保湿面膜" in query:
        return "关于深海蓝藻保湿面膜的用户评价：普遍反馈补水效果好，吸收快，对敏感肌友好。有用户提到价格略高，但效果值得。"
    else:
        return f"未找到关于 '{query}' 的特定信息，但市场反馈通常关注产品成分、功效和用户体验。"

def mock_query_product_database(product_name: str) -> str:
    """模拟查询产品数据库，返回预设的产品信息。"""
    print(f"[Tool Call] 模拟查询产品数据库：{product_name}")
    time.sleep(0.5) # 模拟数据库查询延迟
    if "深海蓝藻保湿面膜" in product_name:
        return "深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。"
    elif "美白精华" in product_name:
        return "美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。"
    else:
        return f"产品数据库中未找到关于 '{product_name}' 的详细信息。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号，根据上下文提供常用表情。"""
    print(f"[Tool Call] 模拟生成表情符号，上下文：{context}")
    time.sleep(0.2) # 模拟生成延迟
    if "补水" in context or "水润" in context or "保湿" in context:
        return ["💦", "💧", "🌊", "✨"]
    elif "惊喜" in context or "哇塞" in context or "爱了" in context:
        return ["💖", "😍", "🤩", "💯"]
    elif "熬夜" in context or "疲惫" in context:
        return ["😭", "😮‍💨", "😴", "💡"]
    elif "好物" in context or "推荐" in context:
        return ["✅", "👍", "⭐", "🛍️"]
    else:
        return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩", "💧", "🌿"], k=min(5, len(context.split())))

# 将模拟工具函数映射到一个字典，方便通过名称调用
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": mock_query_product_database,
    "generate_emoji": mock_generate_emoji,
}

## 4. 实战：构建小红书文案生成 Agent

现在，我们将把 System Prompt、工具定义和模拟工具函数整合起来，构建出能够自动执行的 DeepSeek Agent 工作流。核心是 `generate_rednote` 函数，它通过一个循环来模拟 Agent 的 `Thought-Action-Observation` 过程。

In [5]:
import json
import re

def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 5) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    
    Args:
        product_name (str): 要生成文案的产品名称。
        tone_style (str): 文案的语气和风格，如"活泼甜美"、"知性"、"搞怪"等。
        max_iterations (int): Agent 最大迭代次数，防止无限循环。
        
    Returns:
        str: 生成的爆款文案（JSON 格式字符串）。
    """
    
    print(f"\n🚀 启动小红书文案生成助手，产品：{product_name}，风格：{tone_style}\n")
    
    # 存储对话历史，包括系统提示词和用户请求
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出，并确保JSON内容用markdown代码块包裹（例如：```json{{...}}```）。"}
    ]
    
    iteration_count = 0
    final_response = None
    
    while iteration_count < max_iterations:
        iteration_count += 1
        print(f"-- Iteration {iteration_count} --")
        
        try:
            # 调用 DeepSeek API，传入对话历史和工具定义
            response = client.chat.completions.create(
                model="deepseek-ai/DeepSeek-V3",
                messages=messages,
                tools=TOOLS_DEFINITION, # 告知模型可用的工具
                tool_choice="auto" # 允许模型自动决定是否使用工具
            )

            response_message = response.choices[0].message
            
            # **ReAct模式：处理工具调用**
            if response_message.tool_calls: # 如果模型决定调用工具
                print("Agent: 决定调用工具...")
                messages.append(response_message) # 将工具调用信息添加到对话历史
                
                tool_outputs = []
                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    # 确保参数是合法的JSON字符串，即使工具不要求参数，也需要传递空字典
                    function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}

                    print(f"Agent Action: 调用工具 '{function_name}'，参数：{function_args}")
                    
                    # 查找并执行对应的模拟工具函数
                    if function_name in available_tools:
                        tool_function = available_tools[function_name]
                        tool_result = tool_function(**function_args)
                        print(f"Observation: 工具返回结果：{tool_result}")
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(tool_result) # 工具结果作为字符串返回
                        })
                    else:
                        error_message = f"错误：未知的工具 '{function_name}'"
                        print(error_message)
                        tool_outputs.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": error_message
                        })
                messages.extend(tool_outputs) # 将工具执行结果作为 Observation 添加到对话历史
                
            # **ReAct 模式：处理最终内容**
            elif response_message.content: # 如果模型直接返回内容（通常是最终答案）
                print(f"[模型生成结果] {response_message.content}")
                
                # --- START: 添加 JSON 提取和解析逻辑 ---
                json_string_match = re.search(r"```json\s*(\{.*\})\s*```", response_message.content, re.DOTALL)
                
                if json_string_match:
                    extracted_json_content = json_string_match.group(1)
                    try:
                        final_response = json.loads(extracted_json_content)
                        print("Agent: 任务完成，成功解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError as e:
                        print(f"Agent: 提取到JSON块但解析失败: {e}")
                        print(f"尝试解析的字符串:\n{extracted_json_content}")
                        messages.append(response_message) # 解析失败，继续对话
                else:
                    # 如果没有匹配到 ```json 块，尝试直接解析整个 content
                    try:
                        final_response = json.loads(response_message.content)
                        print("Agent: 任务完成，直接解析最终JSON文案。")
                        return json.dumps(final_response, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        print("Agent: 生成了非JSON格式内容或非Markdown JSON块，可能还在思考或出错。")
                        messages.append(response_message) # 非JSON格式，继续对话
                # --- END: 添加 JSON 提取和解析逻辑 ---
            else:
                print("Agent: 未知响应，可能需要更多交互。")
                break
                
        except Exception as e:
            print(f"调用 DeepSeek API 时发生错误: {e}")
            break
    
    print("\n⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。")
    return "未能成功生成文案。"

## 5. 实际测试与文案生成

现在，让我们调用我们构建的 `generate_rednote` 函数，看看它能生成什么样的爆款文案！

In [6]:
# 测试案例 1: 深海蓝藻保湿面膜
product_name_1 = "深海蓝藻保湿面膜"
tone_style_1 = "活泼甜美"
result_1 = generate_rednote(product_name_1, tone_style_1)

print("\n--- 生成的文案 1 ---")
print(result_1)


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation: 工具返回结果：深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '补水保湿、修护肌肤、深度滋养'}
[Tool Call] 模拟生成表情符号，上下文：补水保湿、修护肌肤、深度滋养
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "💦干皮星人救星！这款深海蓝藻保湿面膜让我爱上敷面膜✨",
  "body": "姐妹们！终于找到一款让我爱不释手的保湿面膜啦！🌊深海蓝藻保湿面膜真的绝绝子！\n\n🌟核心卖点：\n- 深海蓝藻提取物，满满的多糖和氨基酸，深层补水修护！\n- 清爽不粘腻，完全不会闷痘，敏感肌也能放心用！\n- 一片25ml精华液，敷完脸蛋水当当，像剥了壳的鸡蛋！\n\n💧我的使用心得：\n最近换季皮肤超级干燥，还老是泛红，用了这款面膜后真的改善了超多！精华液巨多，敷完还能涂脖子和手臂，一点都不浪费～连续用了5天，皮肤状态直接拉满，连闺蜜都问我是不是去打水光针了！\n\n姐妹们，这款真的是干皮星人和敏感肌的福音！赶紧冲！🔥",
  "hashtags": ["#保湿面膜", "#深海蓝藻", "#干皮救星", "#敏感肌友好", "#护肤好物"],
  "emojis": ["💦", "💧", "🌊", "✨", "🔥"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 1 ---
{
  "title": "💦干皮星人救星！

In [7]:
# 测试案例 2: 美白精华
product_name_2 = "美白精华"
tone_style_2 = "知性温柔"
result_2 = generate_rednote(product_name_2, tone_style_2)

print("\n--- 生成的文案 2 ---")
print(result_2)


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation: 工具返回结果：美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 2 --
Agent: 决定调用工具...
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '提亮肤色、淡化痘印、改善暗沉'}
[Tool Call] 模拟生成表情符号，上下文：提亮肤色、淡化痘印、改善暗沉
Observation: 工具返回结果：['✨']
-- Iteration 3 --
[模型生成结果] ```json
{
  "title": "✨一瓶搞定暗沉痘印！我的透亮美肌秘方分享✨",
  "body": "最近被闺蜜疯狂夸皮肤透亮，问我用了什么神器🥰\n其实是这瓶美白精华拯救了我的暗沉脸！\n\n💡核心成分：烟酰胺+VC衍生物\n✨提亮肤色：用了一周就感觉肤色均匀了好多！\n✨淡化痘印：之前留下的痘印也淡了不少，简直惊喜～\n✨改善暗沉：熬夜党的救星，再也不用担心脸黄了！\n\n质地超级轻薄，吸收快不黏腻，早晚各一次，轻轻按压一下就搞定👌\n\n如果你也需要一瓶能打的美白精华，真的可以试试它！现在每天照镜子都忍不住多看两眼～💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤分享", "#熬夜党救星"],
  "emojis": ["✨", "🥰", "💡", "👌", "💖"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 2 ---
{
  "title": "✨一瓶搞定暗沉痘印！我的透亮美肌秘方分享✨",
  "body": "最近被闺蜜疯狂夸皮肤透亮，问我用了什么神器🥰\n其实是这瓶美白精华拯救了我的暗沉脸！\n\n💡核心成分：烟酰胺+VC衍生物\n✨提亮肤色：用了一周就感觉肤色均匀了好多！

### 格式化 小红书文案

**格式化函数 `format_rednote_for_markdown` 的功能：**

1. 解析 JSON 字符串。
2. 提取标题、正文、标签和表情符号。
3. 将它们组合成一个易读的 Markdown 格式的文本。


**工作方式：**

1. **解析 JSON**：使用 `json.loads()` 将输入的字符串转换为 Python 字典。如果解析失败，会返回一个错误信息。
2. **提取数据**：使用 `.get()` 方法从字典中安全地提取 `title`、`body` 和 `hashtags`。使用 `.get()` 的好处是，如果某个键不存在，它会返回一个默认值（例如 `None` 或空列表），而不是抛出 `KeyError`。
3. **构建 Markdown 标题**：将 `title` 格式化为 Markdown 的二级标题 (`## Title`)。
4. **处理正文**：直接使用 `body`。由于小红书正文中的换行很重要，我们保留它们。
5. **处理 Hashtags**：将 `hashtags` 列表中的每个标签用空格连接起来，形成一行。
6. **表情符号 (Emojis)**：在小红书的实际发布中，表情符号通常已经嵌入在标题和正文中了。这个函数没有单独列出它们，因为这通常不是最终发布格式的一部分。如果需要，可以取消注释相关代码来单独显示它们。
7. **返回结果**：返回拼接好的 Markdown 字符串，并使用 `.strip()` 去除可能存在于末尾的多余空白。

In [8]:
import json

def format_rednote_for_markdown(json_string: str) -> str:
    """
    将 JSON 格式的小红书文案转换为 Markdown 格式，以便于阅读和发布。

    Args:
        json_string (str): 包含小红书文案的 JSON 字符串。
                           预计格式为 {"title": "...", "body": "...", "hashtags": [...], "emojis": [...]}

    Returns:
        str: 格式化后的 Markdown 文本。
    """
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"错误：无法解析 JSON 字符串 - {e}\n原始字符串：\n{json_string}"

    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    # 表情符号通常已经融入标题和正文中，这里可以选择是否单独列出
    # emojis = data.get("emojis", []) 

    # 构建 Markdown 文本
    markdown_output = f"## {title}\n\n" # 标题使用二级标题
    
    # 正文，保留换行符
    markdown_output += f"{body}\n\n"
    
    # Hashtags
    if hashtags:
        hashtag_string = " ".join(hashtags) # 小红书标签通常是空格分隔
        markdown_output += f"{hashtag_string}\n"
        
    # 如果需要，可以单独列出表情符号，但通常它们已经包含在标题和正文中
    # if emojis:
    #     emoji_string = " ".join(emojis)
    #     markdown_output += f"\n使用的表情：{emoji_string}\n"
        
    return markdown_output.strip() # 去除末尾多余的空白

In [9]:
# --- 示例使用 ---
# 假设这是 generate_rednote 函数的输出
generated_json_output = """
{
  "title": "✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟",
  "body": "姐妹们！我终于找到了我的本命美白精华！💖\\n\\n作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩\\n\\n🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！\\n💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！\\n🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！\\n\\n使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～\\n\\n真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖",
  "hashtags": ["#美白精华", "#提亮肤色", "#淡化痘印", "#护肤好物", "#冷白皮"],
  "emojis": ["✨", "💖", "🤩", "💧", "🌿"]
}
"""

# 调用格式化函数
markdown_note = format_rednote_for_markdown(generated_json_output)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

# --- 另一个例子，假设JSON解析失败 ---
invalid_json_output = "{'title': 'Test', 'body': 'This is not valid json'}" # 使用单引号，非法
markdown_error_note = format_rednote_for_markdown(invalid_json_output)
print("\n--- 格式化错误示例 ---")
print(markdown_error_note)


--- 格式化后的小红书文案 (Markdown) ---
## ✨ 28天逆袭冷白皮！这款美白精华让我告别暗沉痘印 🌟

姐妹们！我终于找到了我的本命美白精华！💖

作为一个常年熬夜➕痘印困扰的混油皮，肤色暗沉一直是我的心头大患。直到遇见了这款美白精华，简直打开了新世界的大门！🤩

🌟 核心成分：烟酰胺+VC衍生物，双管齐下，提亮肤色效果绝绝子！
💧 质地轻薄到爆炸，上脸秒吸收，完全不会黏腻，油皮姐妹放心冲！
🌿 用了28天，痘印肉眼可见变淡了，整张脸都透亮了起来，素颜也能打！

使用方法也很简单：早晚洁面后，滴2-3滴在手心，轻轻按压上脸，后续再叠加保湿产品就OK啦～

真心推荐给所有想要均匀肤色、告别暗沉的姐妹！入股不亏！💖

#美白精华 #提亮肤色 #淡化痘印 #护肤好物 #冷白皮

--- 格式化错误示例 ---
错误：无法解析 JSON 字符串 - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
原始字符串：
{'title': 'Test', 'body': 'This is not valid json'}


In [10]:
# 调用格式化函数
markdown_note = format_rednote_for_markdown(result_1)

# 打印结果
print("--- 格式化后的小红书文案 (Markdown) ---")
print(markdown_note)

--- 格式化后的小红书文案 (Markdown) ---
## 💦干皮星人救星！这款深海蓝藻保湿面膜让我爱上敷面膜✨

姐妹们！终于找到一款让我爱不释手的保湿面膜啦！🌊深海蓝藻保湿面膜真的绝绝子！

🌟核心卖点：
- 深海蓝藻提取物，满满的多糖和氨基酸，深层补水修护！
- 清爽不粘腻，完全不会闷痘，敏感肌也能放心用！
- 一片25ml精华液，敷完脸蛋水当当，像剥了壳的鸡蛋！

💧我的使用心得：
最近换季皮肤超级干燥，还老是泛红，用了这款面膜后真的改善了超多！精华液巨多，敷完还能涂脖子和手臂，一点都不浪费～连续用了5天，皮肤状态直接拉满，连闺蜜都问我是不是去打水光针了！

姐妹们，这款真的是干皮星人和敏感肌的福音！赶紧冲！🔥

#保湿面膜 #深海蓝藻 #干皮救星 #敏感肌友好 #护肤好物


## 7. 总结与展望

通过本次实战，我们成功构建了一个基于 DeepSeek Agent 的小红书爆款文案生成助手。我们学习了如何拆解需求、设计 Prompt、定义工具，并实现 Agent 的核心工作流。

Agent 在内容营销领域的潜力巨大，未来可以进一步拓展到：

*   **超个性化内容：** 根据用户数据，生成一对一的定制文案。
*   **多模态内容创作：** 结合图片、视频生成，实现图文音视频一体化。
*   **智能营销决策：** Agent 不仅生成内容，还能分析效果并给出投放建议。
*   **跨平台适配：** 快速生成适应不同社交媒体平台风格的文案。

同时，我们也需关注挑战，如确保内容真实性、处理高度主观情感、与现有工作流的无缝集成等。Agent 技术仍在快速发展，期待未来能带来更多惊喜！

# HOMOWORK

## 三种不同风格主题文案

In [11]:
# 测试案例 1: 美白精华
product_name_3 = "美白精华"
tone_style_3 = "知性温柔"
result_3 = generate_rednote(product_name_3, tone_style_3)

print("\n--- 生成的文案 test 1 (Markdone)---")
print(format_rednote_for_markdown(result_3))


🚀 启动小红书文案生成助手，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '美白精华'}
[Tool Call] 模拟查询产品数据库：美白精华
Observation: 工具返回结果：美白精华：核心成分是烟酰胺和VC衍生物，主要功效是提亮肤色、淡化痘印、改善暗沉。质地轻薄易吸收，适合需要均匀肤色的人群。
-- Iteration 2 --
Agent: 未知响应，可能需要更多交互。

⚠️ Agent 达到最大迭代次数或未能生成最终文案。请检查Prompt或增加迭代次数。

--- 生成的文案 test 1 (Markdone)---
错误：无法解析 JSON 字符串 - Expecting value: line 1 column 1 (char 0)
原始字符串：
未能成功生成文案。


In [12]:
# 测试案例 2: 深海蓝藻保湿面膜
product_name_4 = "深海蓝藻保湿面膜"
tone_style_4 = "搞怪"
result_4 = generate_rednote(product_name_4, tone_style_4)

print("\n--- 生成的文案 test 2 (Markdone)---")
print(format_rednote_for_markdown(result_4))


🚀 启动小红书文案生成助手，产品：深海蓝藻保湿面膜，风格：搞怪

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '深海蓝藻保湿面膜'}
[Tool Call] 模拟查询产品数据库：深海蓝藻保湿面膜
Observation: 工具返回结果：深海蓝藻保湿面膜：核心成分为深海蓝藻提取物，富含多糖和氨基酸，能深层补水、修护肌肤屏障、舒缓敏感泛红。质地清爽不粘腻，适合所有肤质，尤其适合干燥、敏感肌。规格：25ml*5片。
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '深海蓝藻保湿面膜的搞怪风格'}
[Tool Call] 模拟生成表情符号，上下文：深海蓝藻保湿面膜的搞怪风格
Observation: 工具返回结果：['💦', '💧', '🌊', '✨']
-- Iteration 2 --
[模型生成结果] ```json
{
  "title": "震惊！我的脸竟然被海藻绑架了？🌊✨深海蓝藻保湿面膜测评",
  "body": "姐妹们！我刚敷完一片深海蓝藻保湿面膜，整个人都飘了！💦\n\n这款面膜简直是我的‘补水救星’，它的核心成分是深海蓝藻提取物，富含多糖和氨基酸，敷完脸水润得像刚泡完澡的美人鱼🧜‍♀️（别笑，是真的）！\n\n✨优点：\n1️⃣ 质地清爽不粘腻，敷完完全不闷痘！\n2️⃣ 深层补水，沙漠皮变水嫩皮不是梦！\n3️⃣ 修护肌肤屏障，敏感肌也能安心用！\n\n搞笑插播：\n我妈问我是不是偷用了她的‘神仙水’，笑死！明明是这片海藻面膜把我整容了！😂\n\n总之，姐妹们赶紧冲！你们的钱包会感谢我！（也可能恨我？）💖",
  "hashtags": ["#深海蓝藻保湿面膜", "#补水神器", #面膜测评", "#敏感肌福音", "#搞怪护肤"],
  "emojis": ["💦", "💧", "🌊", "✨", "💖"]
}
```
Agent: 提取到JSON块但解析失败: Expecting value: line 4 column 38 (char 349)
尝试解析的字符串:
{
  "title": "震惊！我的脸竟然被海藻绑架了？🌊✨深海蓝

In [13]:
# 测试案例 3: SK-II精华液
product_name_5 = "SK-II精华液"
tone_style_5 = "活泼甜美"
result_5 = generate_rednote(product_name_5, tone_style_5)

print("\n--- 生成的文案 test 3 (Markdone)---")
print(format_rednote_for_markdown(result_5))


🚀 启动小红书文案生成助手，产品：SK-II精华液，风格：活泼甜美

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': 'SK-II精华液'}
[Tool Call] 模拟查询产品数据库：SK-II精华液
Observation: 工具返回结果：产品数据库中未找到关于 'SK-II精华液' 的详细信息。
Agent Action: 调用工具 'generate_emoji'，参数：{'context': 'SK-II精华液神仙水 护肤 美白 抗衰老'}
[Tool Call] 模拟生成表情符号，上下文：SK-II精华液神仙水 护肤 美白 抗衰老
Observation: 工具返回结果：['🔥', '💖', '✨', '💧']
-- Iteration 2 --
[模型生成结果] ```json
{
  "title": "✨SK-II神仙水精华液✨ 一瓶拯救熬夜肌，肌肤透亮到发光！💖",
  "body": "姐妹们！今天要和大家分享我的心头爱——SK-II神仙水精华液！🔥\n\n🌟 为什么它这么火？因为真的太！好！用！了！\n\n💧 质地清爽不黏腻，吸收超级快，涂完瞬间肌肤喝饱水的感觉～\n💖 坚持用下来，我的熬夜暗沉肌真的透亮了好多！毛孔也细腻了！\n✨ 据说里面还有PITERA™成分，抗老修复一把抓，简直是肌肤的‘神仙水’！\n\n🔍 使用方法也很简单：洁面后，滴几滴在手心，轻轻按压上脸，后续再叠加护肤品，效果翻倍！\n\n用了它，连素颜都敢出门了！姐妹们冲鸭！💃",
  "hashtags": ["#SKII神仙水", "#护肤必备", "#抗衰老神器", "#美白精华", "#熬夜救星"],
  "emojis": ["✨", "🔥", "💖", "💧", "🌟"]
}
```
Agent: 任务完成，成功解析最终JSON文案。

--- 生成的文案 test 3 (Markdone)---
## ✨SK-II神仙水精华液✨ 一瓶拯救熬夜肌，肌肤透亮到发光！💖

姐妹们！今天要和大家分享我的心头爱——SK-II神仙水精华液！🔥

🌟 为什么它这么火？因为真的太！好！用！了！

💧 质地清爽不黏腻，吸收超级快，

# mock_query_product_database修改为RAG 方式

In [14]:
# 数据导入
from glob import glob

text_lines = []

for file_path in glob("./goods_info.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("《")

In [15]:
len(text_lines)

4

In [16]:
# 嵌入模型准备
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='BAAI/bge-m3', # Specify the model name
    api_key=api_key, # Provide your OpenAI API key
    base_url='https://api.siliconflow.cn/v1',
    dimensions=512
)

/home/chj/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# 嵌入模型测试
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

1024
[-0.01914445  0.02160134 -0.0458619  -0.01061761 -0.01767996 -0.01904811
  0.03992683  0.03857796  0.0173042   0.00452116]


In [18]:
# 创建collection

from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

/home/chj/miniconda3/envs/deepseek/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [19]:
# rag 数据插入
from tqdm import tqdm

data = []

def chunked(list_, batch_size):
    for i in range(0, len(list_), batch_size):
        yield list_[i:i+batch_size]

batch_size = 64
all_embeddings = []
for chunk in chunked(text_lines, batch_size):
    emb = embedding_model.encode_documents(chunk)
    all_embeddings.extend(emb)

# doc_embeddings = embedding_model.encode_documents(text_lines_64)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": all_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 4/4 [00:00<00:00, 48770.98it/s]


{'insert_count': 4, 'ids': [0, 1, 2, 3], 'cost': 0}

In [24]:
# 创建查询函数

def get_context(question: str) -> str:
    # 获取上下文
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries(
            [question]
        ),  # 将问题转换为嵌入向量
        limit=3,  # 返回前3个结果
        search_params={"metric_type": "IP", "params": {}},  # 内积距离
        output_fields=["text"],  # 返回 text 字段
    )

    retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]

    context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
    )
    return context

def gen_prompt(question: str, context: str) -> list:
    SYSTEM_PROMPT = """
    Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
    """
    USER_PROMPT = f"""
    请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。对最后输出的内容进行精炼与总结，格式为文本形式。
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    """

    prompt = []
    prompt.append(SYSTEM_PROMPT)
    prompt.append(USER_PROMPT)
    return prompt


def query_goods_info(product_name: str) -> str:
    """查询产品RAG库，返回产品信息。"""
    print(f"[Tool Call] 查询产品RAG库：{product_name}")

    # 构建问题与上下文
    question = f"产品名称：{product_name} 的主要成分以及功效宣称是什么？"
    context = get_context(question=question)
    prompt = gen_prompt(question=question,context=context)

    # 调用模型回答
    response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[
        {"role": "system", "content": prompt[0]},
        {"role": "user", "content": prompt[1]},
    ],
    )
    print(response.choices[0].message.content)

    if "未找到" in response.choices[0].message.content: 
        return 
    else:
        return response.choices[0].message.content

In [25]:
# 修改函数表
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": query_goods_info,
    "generate_emoji": mock_generate_emoji,
}

In [26]:
# 测试案例 1: 雅诗兰黛DW持妆粉底液
product_name_rag = "雅诗兰黛DW持妆粉底液"
tone_style_rag = "知性温柔"
result_rag = generate_rednote(product_name_rag, tone_style_rag)

print("\n--- 生成的文案 rag 1 (Markdone)---")
print(format_rednote_for_markdown(result_rag))


🚀 启动小红书文案生成助手，产品：雅诗兰黛DW持妆粉底液，风格：知性温柔

-- Iteration 1 --
Agent: 决定调用工具...
Agent Action: 调用工具 'query_product_database'，参数：{'product_name': '雅诗兰黛DW持妆粉底液'}
[Tool Call] 查询产品RAG库：雅诗兰黛DW持妆粉底液
雅诗兰黛DW持妆粉底液的主要成分包括聚二甲基硅氧烷等成膜剂、多种调色色素以及透明质酸钠等保湿成分。  
功效宣称：高遮瑕力、长效持妆、哑光妆效，具备防水防汗功能，适合油性和混合性肌肤，可均匀肤色并遮盖瑕疵。
Observation: 工具返回结果：雅诗兰黛DW持妆粉底液的主要成分包括聚二甲基硅氧烷等成膜剂、多种调色色素以及透明质酸钠等保湿成分。  
功效宣称：高遮瑕力、长效持妆、哑光妆效，具备防水防汗功能，适合油性和混合性肌肤，可均匀肤色并遮盖瑕疵。
Agent Action: 调用工具 'generate_emoji'，参数：{'context': '雅诗兰黛DW持妆粉底液的持妆效果与高级感'}
[Tool Call] 模拟生成表情符号，上下文：雅诗兰黛DW持妆粉底液的持妆效果与高级感
Observation: 工具返回结果：['💧']
-- Iteration 2 --
[模型生成结果] ```json
{
  "title": "雅诗兰黛DW持妆粉底液｜一整天不脱妆的秘密✨",
  "body": "姐妹们！今天一定要分享这款让我惊艳的雅诗兰黛DW持妆粉底液💖\n\n🌟 **持妆力Max**：早上化完妆到晚上回家，妆感依然在线！防水防汗的设计简直是为夏天量身定制～\n\n🌟 **高遮瑕力**：毛孔、痘印统统隐形，一键磨皮的哑光妆感真的太高级了！\n\n🌟 **适合油皮/混油**：控油效果杠杠的，再也不用担心T区出油脱妆了💧\n\n🌟 **保湿不拔干**：透明质酸钠的加持，妆面服帖又自然，一点都不会觉得紧绷哦～\n\n如果你也在找一款既能遮瑕又能持久的粉底液，DW绝对是你的不二之选🔥",
  "hashtags": ["#雅诗兰黛DW粉底液", "#持妆粉底液", "#油皮救星", "#高级哑光妆感", "#美妆好物推荐"],
  "emojis": ["✨", "💖", "💧", "🌟", "🔥